Дотасет:
https://www.kaggle.com/c/kkbox-music-recommendation-challenge/overview

Можете мерять результат по 5-fold evaluation на train-set,  per user auc roc.  

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, average_precision_score
from catboost import Pool, CatBoostClassifier
import os
from os.path import join
from tqdm.notebook import tqdm
from collections import defaultdict
from sklearn.model_selection import train_test_split

In [2]:
DATA_DIR = './data'
GLOBAL_SEED = 0
K_FOLDS = 5

# Задача 1:

Собрать более-менее приличную рекомендательную систему с помощью GBDT. Использование трюков из лекции (и не из неё), тюнинг и разумных объемов фича инжиниринг прям нужен. 

## EDA

In [3]:
train_df_raw = pd.read_csv(join(DATA_DIR, "train.csv"))
train_df_raw.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,target
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1


In [4]:
train_df_raw.dtypes

msno                  object
song_id               object
source_system_tab     object
source_screen_name    object
source_type           object
target                 int64
dtype: object

In [5]:
train_df_raw.isna().sum(axis=0)

msno                       0
song_id                    0
source_system_tab      24849
source_screen_name    414804
source_type            21539
target                     0
dtype: int64

In [6]:
train_df_raw.isna().sum(axis=0) / len(train_df_raw)

msno                  0.000000
song_id               0.000000
source_system_tab     0.003368
source_screen_name    0.056226
source_type           0.002920
target                0.000000
dtype: float64

In [7]:
train_df_raw.fillna('nan', inplace=True)

In [8]:
train_df_raw.nunique()

msno                   30755
song_id               359966
source_system_tab          9
source_screen_name        21
source_type               13
target                     2
dtype: int64

In [9]:
songs_df = pd.read_csv(join(DATA_DIR, 'songs.csv'))
songs_df.head()

,song_id,song_length,genre_ids,artist_name,composer,lyricist,language
0,CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=,247640,465,張信哲 (Jeff Chang),董貞,何啟弘,3.0
1,o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=,197328,444,BLACKPINK,TEDDY| FUTURE BOUNCE| Bekuh BOOM,TEDDY,31.0
2,DwVvVurfpuz+XPuFvucclVQEyPqcpUkHR0ne1RQzPs0=,231781,465,SUPER JUNIOR,NaN,NaN,31.0
3,dKMBWoZyScdxSkihKG+Vf47nc18N9q4m58+b4e7dSSE=,273554,465,S.H.E,湯小康,徐世珍,3.0
4,W3bqWd3T+VeHFzHAUfARgW9AvVRaF4N5Yzm4Mr6Eo/o=,140329,726,貴族精選,Traditional,Traditional,52.0


In [10]:
songs_df.isna().sum(axis=0) / len(songs_df)

song_id        0.000000e+00
song_length    0.000000e+00
genre_ids      4.098558e-02
artist_name    0.000000e+00
composer       4.665526e-01
lyricist       8.471241e-01
language       4.354794e-07
dtype: float64

In [11]:
songs_df.dtypes

song_id         object
song_length      int64
genre_ids       object
artist_name     object
composer        object
lyricist        object
language       float64
dtype: object

In [12]:
songs_df.isna().sum(axis=0) / len(songs_df)

song_id        0.000000e+00
song_length    0.000000e+00
genre_ids      4.098558e-02
artist_name    0.000000e+00
composer       4.665526e-01
lyricist       8.471241e-01
language       4.354794e-07
dtype: float64

In [13]:
train_df = train_df_raw.merge(songs_df, on='song_id', how='left')

In [14]:
members_df = pd.read_csv(join(DATA_DIR, 'members.csv'))
members_df.head()

,msno,city,bd,gender,registered_via,registration_init_time,expiration_date
0,XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=,1,0,NaN,7,20110820,20170920
1,UizsfmJb9mV54qE9hCYyU07Va97c0lCRLEQX3ae+ztM=,1,0,NaN,7,20150628,20170622
2,D8nEhsIOBSoE6VthTaqDX8U6lqjJ7dLdr72mOyLya2A=,1,0,NaN,4,20160411,20170712
3,mCuD+tZ1hERA/o5GPqk38e041J8ZsBaLcu7nGoIIvhI=,1,0,NaN,9,20150906,20150907
4,q4HRBfVSssAFS9iRfxWrohxuk9kCYMKjHOEagUMV6rQ=,1,0,NaN,4,20170126,20170613


In [15]:
members_df.dtypes

msno                      object
city                       int64
bd                         int64
gender                    object
registered_via             int64
registration_init_time     int64
expiration_date            int64
dtype: object

In [16]:
members_df.isna().sum(axis=0) / len(members_df)

msno                      0.000000
city                      0.000000
bd                        0.000000
gender                    0.578496
registered_via            0.000000
registration_init_time    0.000000
expiration_date           0.000000
dtype: float64

In [17]:
members_df.fillna('nan', inplace=True)

In [18]:
def split_year_month_day(dt):
    return dt // 10000, (dt % 10000) // 100, dt % 100

In [19]:
members_df['reg_year'], members_df['reg_month'], members_df['reg_day'] = zip(
    *members_df.registration_init_time.map(split_year_month_day)
)

In [20]:
members_df['exp_year'], members_df['exp_month'], members_df['exp_day'] = zip(
    *members_df.expiration_date.map(split_year_month_day)
)

In [21]:
members_df.drop(labels=['registration_init_time', 'expiration_date'], axis=1, inplace=True)
members_df.columns

Index(['msno', 'city', 'bd', 'gender', 'registered_via', 'reg_year',
       'reg_month', 'reg_day', 'exp_year', 'exp_month', 'exp_day'],
      dtype='object')

In [22]:
members_df.bd.loc[(members_df.bd < 5) | (members_df.bd > 100)] = np.nan

/home/anton/python-envs/recsys-env/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [23]:
train_df = train_df.merge(members_df, on='msno', how='left')

In [24]:
train_df.dtypes

msno                   object
song_id                object
source_system_tab      object
source_screen_name     object
source_type            object
target                  int64
song_length           float64
genre_ids              object
artist_name            object
composer               object
lyricist               object
language              float64
city                    int64
bd                    float64
gender                 object
registered_via          int64
reg_year                int64
reg_month               int64
reg_day                 int64
exp_year                int64
exp_month               int64
exp_day                 int64
dtype: object

In [25]:
train_df.language.unique()

array([52., -1., 31.,  3., 17., 10., 24., 59., 45., nan, 38.])

In [26]:
train_df.language = train_df.language.fillna(0).astype('int64')

In [27]:
train_df.dtypes

msno                   object
song_id                object
source_system_tab      object
source_screen_name     object
source_type            object
target                  int64
song_length           float64
genre_ids              object
artist_name            object
composer               object
lyricist               object
language                int64
city                    int64
bd                    float64
gender                 object
registered_via          int64
reg_year                int64
reg_month               int64
reg_day                 int64
exp_year                int64
exp_month               int64
exp_day                 int64
dtype: object

Заменим набор жанров на эмпирическую вероятность того, что юзер послушает песню с данным набором жанров (в предположении, что жанры независимы)

In [28]:
user_genre_counter = dict((msno, defaultdict(int)) for msno in train_df.msno.unique().tolist())

In [29]:
for msno, genre_ids in zip(train_df.msno, train_df.genre_ids):
    if pd.notna(genre_ids):
        user_genre_counter[msno]['num_songs_listened'] += 1
        for genre in genre_ids.split('|'):
            user_genre_counter[msno][genre] += 1

In [30]:
user_genre_counter['FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=']

defaultdict(int,
            {'num_songs_listened': 5410,
             '359': 182,
             '1011': 39,
             '1259': 674,
             '465': 2369,
             '1609': 1042,
             '139': 190,
             '691': 2,
             '2022': 194,
             '1616': 58,
             '2058': 13,
             '2072': 4,
             '125': 60,
             '109': 62,
             '444': 201,
             '1955': 3,
             '352': 11,
             '1995': 11,
             '921': 112,
             '1040': 1,
             '2093': 6,
             '423': 2,
             '458': 281,
             '451': 28,
             '430': 11,
             '437': 8,
             '2122': 37,
             '880': 2,
             '118': 1,
             '2107': 15,
             '2176': 4,
             '1572': 10,
             '275': 10,
             '829': 4,
             '958': 18,
             '2130': 1,
             '798': 13,
             '2157': 11,
             '947': 12,
             '

In [31]:
user_genre_logprobs = []
for msno, genre_ids in zip(train_df.msno, train_df.genre_ids):
    if pd.isna(genre_ids):
        user_genre_logprobs.append(np.nan)
    else:
        user_genre_logprobs.append(
            np.log([
                user_genre_counter[msno][genre] / user_genre_counter[msno]['num_songs_listened']
            for genre in genre_ids.split('|')]).sum()
        )
train_df['genres_logprob'] = np.array(user_genre_logprobs)

In [32]:
train_df.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,target,song_length,genre_ids,artist_name,composer,...,bd,gender,registered_via,reg_year,reg_month,reg_day,exp_year,exp_month,exp_day,genres_logprob
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1,206471.0,359,Bastille,Dan Smith| Mark Crew,...,NaN,nan,7,2012,1,2,2017,10,5,-3.391998
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1,284584.0,1259,Various Artists,NaN,...,24.0,female,9,2011,5,25,2017,9,11,-2.528175
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1,225396.0,1259,Nas,N. Jones、W. Adams、J. Lordan、D. Ingle,...,24.0,female,9,2011,5,25,2017,9,11,-2.528175
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1,255512.0,1019,Soundway,Kwadwo Donkoh,...,24.0,female,9,2011,5,25,2017,9,11,-6.419995
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1,187802.0,1011,Brett Young,Brett Young| Kelly Archer| Justin Ebach,...,NaN,nan,7,2012,1,2,2017,10,5,-4.932443


In [33]:
del user_genre_counter, user_genre_logprobs

Сделаем то же самое с исполнителем (на композитора и автора текста забьем)

In [34]:
user_artist_counter = dict((msno, defaultdict(int)) for msno in train_df.msno.unique().tolist())
for msno, artist in zip(train_df.msno, train_df.artist_name):
    if pd.notna(artist):
        user_artist_counter[msno]['num_songs_listened'] += 1
        user_artist_counter[msno][artist] += 1
        
user_artist_logprobs = []
for msno, artist in zip(train_df.msno, train_df.artist_name):
    if pd.isna(artist):
        user_artist_logprobs.append(np.nan)
    else:
        user_artist_logprobs.append(
            np.log(user_artist_counter[msno][artist] / user_artist_counter[msno]['num_songs_listened'])
        )
train_df['artist_logprob'] = np.array(user_artist_logprobs)

del user_artist_counter, user_artist_logprobs

In [35]:
train_df.dtypes

msno                   object
song_id                object
source_system_tab      object
source_screen_name     object
source_type            object
target                  int64
song_length           float64
genre_ids              object
artist_name            object
composer               object
lyricist               object
language                int64
city                    int64
bd                    float64
gender                 object
registered_via          int64
reg_year                int64
reg_month               int64
reg_day                 int64
exp_year                int64
exp_month               int64
exp_day                 int64
genres_logprob        float64
artist_logprob        float64
dtype: object

In [36]:
y = train_df.target
X = train_df.drop(
    ['msno', 'song_id', 'target', 'genre_ids', 'artist_name', 'composer', 'lyricist'],
    axis=1
)

X.dtypes

source_system_tab      object
source_screen_name     object
source_type            object
song_length           float64
language                int64
city                    int64
bd                    float64
gender                 object
registered_via          int64
reg_year                int64
reg_month               int64
reg_day                 int64
exp_year                int64
exp_month               int64
exp_day                 int64
genres_logprob        float64
artist_logprob        float64
dtype: object

In [37]:
del train_df_raw, songs_df, members_df

In [38]:
CAT_FEATURES = [0, 1, 2, 4, 5, 7, 8]

In [39]:
MODEL_PARAMS = {
    'iterations': 100,
    'depth': 8,
    'num_leaves': 256,
    'min_data_in_leaf': 10,
    'random_seed': GLOBAL_SEED,
    'learning_rate': 0.4,
    'eval_metric': 'AUC',
    'early_stopping_rounds': 10,
}

In [40]:
def cross_validation(X, y, k_folds):
    np.random.seed(GLOBAL_SEED)
    idx = np.random.permutation(len(y))
    folds = np.array_split(idx, k_folds)
  
    aucs = []
    
    for split, val_inds in enumerate(folds):
        print(f"Taking {split} split as validation set")
        train_inds = np.setdiff1d(idx, val_inds)

        X_train, y_train = X.iloc[train_inds], y.iloc[train_inds]
        X_val, y_val = X.iloc[val_inds], y.iloc[val_inds]
        
        model = CatBoostClassifier(**MODEL_PARAMS)
        model.fit(
            X_train, y_train,
            cat_features=CAT_FEATURES,
            eval_set=(X_val, y_val),
            verbose=True
        )
        
        probs = model.predict_proba(X_val)[:, 1]
        
        aucs.append(roc_auc_score(y_val.values, probs))
        del model
        
    return aucs

In [42]:
idx = np.random.permutation(len(X))
split = int(len(idx) * 0.8)
train_idx, val_idx = idx[:split], idx[split:]
X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]


model = CatBoostClassifier(**MODEL_PARAMS)
model.fit(
    X_train, y_train,
    cat_features=CAT_FEATURES,
    eval_set=(X_val, y_val),
    verbose=True
)

probs = model.predict_proba(X_val)[:, 1]

print("ROC-AUC", roc_auc_score(y_val.values, probs))

0:	test: 0.6559452	best: 0.6559452 (0)	total: 2.75s	remaining: 4m 32s
1:	test: 0.6610996	best: 0.6610996 (1)	total: 5.4s	remaining: 4m 24s
2:	test: 0.6624602	best: 0.6624602 (2)	total: 7.53s	remaining: 4m 3s
3:	test: 0.6640409	best: 0.6640409 (3)	total: 9.3s	remaining: 3m 43s
4:	test: 0.6659418	best: 0.6659418 (4)	total: 11.6s	remaining: 3m 40s
5:	test: 0.6682752	best: 0.6682752 (5)	total: 13.9s	remaining: 3m 38s
6:	test: 0.6703118	best: 0.6703118 (6)	total: 16.1s	remaining: 3m 33s
7:	test: 0.6713928	best: 0.6713928 (7)	total: 18.4s	remaining: 3m 31s
8:	test: 0.6720349	best: 0.6720349 (8)	total: 20.4s	remaining: 3m 26s
9:	test: 0.6729906	best: 0.6729906 (9)	total: 22.1s	remaining: 3m 18s
10:	test: 0.6742064	best: 0.6742064 (10)	total: 24.3s	remaining: 3m 16s
11:	test: 0.6758312	best: 0.6758312 (11)	total: 26.2s	remaining: 3m 11s
12:	test: 0.6765373	best: 0.6765373 (12)	total: 28.2s	remaining: 3m 8s
13:	test: 0.6769705	best: 0.6769705 (13)	total: 30s	remaining: 3m 4s
14:	test: 0.6773629

Кросс валидация почему-то по памяти пока падет, так что пока так :)

# Задача 2: 
Пользуясь информацией о треках построить неклассические эмбеддинги и попробовать использовать их:

<ol>
    <li>показать симилары глазами</li>
    <li>в качестве рекоммендера вместо GBDT (разницу в метриках)</li> 
    <li>стек с GBDT (прирост / отсутствие / почему /, SHAP values)</li>
</ol>

In [ ]:
#TODO